# Finetuning a custom question answering model
The [BERT](https://arxiv.org/abs/1810.04805) family of models are a powerful set of natural language understanding models based on the transformer architecture originating from the paper Attention Is All You Need, which you can find [here](https://arxiv.org/abs/1706.03762).  

These models work by running unsupervised pre-training on massive sets of text data. This process requires an enormous amount of time and compute. Luckily for us, BERT models are built for transfer learning. BERT models are able to be finetuned to perform many different NLU tasks like question answering, sentiment analysis, document summarization, and more.

For this tutorial, we are going to download the [Stanford Question Answering Dataset](https://rajpurkar.github.io/SQuAD-explorer/) and walk through the  steps necessary to augment it with our own questions using [SageMaker Ground Truth](https://aws.amazon.com/sagemaker/groundtruth/), use [Huggingface](https://huggingface.co/) to finetune a BERT variant for question answering, deploy our finetuned model to a Sagemaker endpoint, and then visualize some results.

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import collections
import math
import random
import torch
import os, tarfile, json
import time, datetime
from io import StringIO
import numpy as np
import boto3
import sagemaker
from tqdm import tqdm
from IPython.display import Markdown as md
from sagemaker.pytorch import estimator, PyTorchModel, PyTorchPredictor, PyTorch
from sagemaker.utils import name_from_base
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline, TrainingArguments, Trainer, default_data_collator
from sagemaker.huggingface import HuggingFace, HuggingFaceModel
import datasets
from datasets import load_dataset, load_metric, Dataset, Features

# Configuration file with IAM role ARN, bucket name, and pre/post processing lambda ARN.
CONFIG_FILE_PATH = "~/SageMaker/gt-hf-squad-config.json"

sagemaker_cl = boto3.client("sagemaker")
sagemaker_session = sagemaker.Session()
s3 = boto3.resource("s3")
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
prefix = 'hf_squad'
runtime_client = boto3.client('runtime.sagemaker')

with open(CONFIG_FILE_PATH) as f:
    config = json.load(f)

role_arn = config["SageMakerRoleArn"]
bucket = config['LabelingJobInputBucket']
pre_annot_lambda_arn = config["PreLabelTaskLambdaArn"]
post_annot_lambda_arn = config["PostLabelTaskLambdaArn"]
input_bucket_name = config["LabelingJobInputBucket"]

## Prerequisites

You will create some of the resources you need to launch a Ground Truth labeling job in this notebook. You must create the following resources before executing this notebook:

* A work team. A work team is a group of workers that complete labeling tasks. If you want to preview the worker UI and execute the labeling task you will need to create a private work team, add yourself as a worker to this team, and provide the work team ARN below. This [GIF](images/create-workteam-loop.gif) demonstrates how to quickly create a private work team on the Amazon SageMaker console. If you do not want to use a private or vendor work team ARN, set `private_work_team` to `False` to use the Amazon Mechanical Turk workforce. To learn more about private, vendor, and Amazon Mechanical Turk workforces, see [Create and Manage Workforces
](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html).

In [248]:
WORKTEAM_ARN = '<<ADD WORK TEAM ARN HERE>>'

print(f'This notebook will use the work team ARN: {WORKTEAM_ARN}')

# Make sure workteam arn is populated if private work team is chosen
assert (WORKTEAM_ARN != '<<ADD WORK TEAM ARN HERE>>')

This notebook will use the work team ARN: arn:aws:sagemaker:us-east-1:322552456788:workteam/private-crowd/internal-test


## Download and inspect the data

Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

To get an idea of what SQuAD contains, let's download it locally and take a look.

SQuAD was created by Pranav Rajpurkar, Jian Zhang, Konstantin Lopyrev, and Percy Liang. You can find the original paper [here](https://arxiv.org/abs/1606.05250) and the dataset [here](https://rajpurkar.github.io/SQuAD-explorer/). SQuAD has been licensed by the authors under the [Creative Commons Attribution-ShareAlike 4.0 International Public License](https://creativecommons.org/licenses/by-sa/4.0/legalcode)


In [ ]:
%%sh 

mkdir data
    
v2="data/v2.0"
mkdir $v2
wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O $v2/train-v2.0.json
wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O $v2/dev-v2.0.json

In [6]:
# load the v2.0 dev set

with open('data/v2.0/dev-v2.0.json', 'r') as f:
    squad_dev = json.load(f)

Now that we've loaded some of the data, you can use the following block to look at a random context, question, and answer:

In [8]:
# view a random Q&A pair
ind = random.randint(0,34)
sq = squad_dev['data'][ind]
print('Paragraph title: ',sq['title'], '\n')
print(sq['paragraphs'][0]['context'],'\n')
print('Question:', sq['paragraphs'][0]['qas'][0]['question'])
print('Answer:', sq['paragraphs'][0]['qas'][0]['answers'][0]['text'])


Paragraph title:  Computational_complexity_theory 

Computational complexity theory is a branch of the theory of computation in theoretical computer science that focuses on classifying computational problems according to their inherent difficulty, and relating those classes to each other. A computational problem is understood to be a task that is in principle amenable to being solved by a computer, which is equivalent to stating that the problem may be solved by mechanical application of mathematical steps, such as an algorithm. 

Question: What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?
Answer: Computational complexity theory


## Load model

Now that we've viewed some of the different question and answer pairs in SQuAD, let's download a model that we can finetune for question answering. Huggingface allows us to easily download a base model that has undergone large scale pre-training and re-initialize it for a different downstream task. In this case we are taking the distilbert-base-uncased model and repurposing it for question answering using the AutoModelForQuestionAnswering class from Huggingface. We also utilize the AutoTokenizer class to retrieve the model's pre-trained tokenizer. Since the models are PyTorch based, we can look at the different modules in the model, the following block will print out the question answering output layer after downloading the model and tokenizer.

In [ ]:
# loadbase_model_prefix 
model_name = "distilbert-base-uncased"

# Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# set model to evaluation mode
evl = model.eval()
print(' \nOutput layer:',list(model.named_modules())[-2])

In [ ]:
# view model layers
model

## View BERT input

BERT needs us to transform our text data into a numeric representation known as tokens. There are a variety of tokenizers available, we are going to use a tokenizer specially designed for BERT that we will instantiate with our vocabulary file. Let's take a look at our transformed question and context we will be supplying BERT for inference.

In [11]:
QA_input = {
    'question': sq['paragraphs'][0]['qas'][0]['question'],
    'context': sq['paragraphs'][0]['context']
}
inputs = tokenizer(QA_input['question'], QA_input['context'], return_tensors='pt')
input_ids = inputs["input_ids"].tolist()[0]
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])
inputs

{'input_ids': tensor([[  101,  2054,  3589,  1997,  9373,  3274,  2671,  9144,  2007, 13644,
         26268,  2075, 15078,  3471,  2011,  7669,  1998,  2465,  1997,  3276,
          1029,   102, 15078, 11619,  3399,  2003,  1037,  3589,  1997,  1996,
          3399,  1997, 22334,  1999,  9373,  3274,  2671,  2008,  7679,  2006,
         26268,  2075, 15078,  3471,  2429,  2000,  2037, 16112,  7669,  1010,
          1998,  8800,  2216,  4280,  2000,  2169,  2060,  1012,  1037, 15078,
          3291,  2003,  5319,  2000,  2022,  1037,  4708,  2008,  2003,  1999,
          6958,  2572,  8189,  3468,  2000,  2108, 13332,  2011,  1037,  3274,
          1010,  2029,  2003,  5662,  2000,  5517,  2008,  1996,  3291,  2089,
          2022, 13332,  2011,  6228,  4646,  1997,  8045,  4084,  1010,  2107,
          2004,  2019,  9896,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Model inference

Let's try getting some predictions from our question answering model.

In [14]:
outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)

answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits
answer_start = torch.argmax(
    answer_start_scores
)  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
# convert answers back to english
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Question: {QA_input['question']}")
print(f"Answer: {answer}")

Question: What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?
Answer: 


## Augment SQuAD

It looks like we didn't get any output from the model for our previous prediction. This is because the weights for the question answering head have not been tuned. In order to tune them we need to train the model on our question answering dataset, but what if we want the model to perform particularly well on our domain specific questions? While it might be able to generalize by simply training on SQuAD, we can augment our training or validation set with our own questions. 

We can use the following code to augment SQuAD by launching a labeling job using a custom question answering template through SageMaker Ground Truth.

In [ ]:
print(f"Uploading annotation UI to {input_bucket_name}")
annotation_ui_s3_uri = f"s3://{input_bucket_name}/{annotation_ui_name}"
!aws s3 cp qa_annotator.liquid.html {annotation_ui_s3_uri}

squad_s3_uri = f"s3://{input_bucket_name}/data/sample_squad.json"
!aws s3 cp sample_squad.json {squad_s3_uri}

# create our manifest
sample_manifest_lines = [{"source": squad_s3_uri}]
sample_manifest = "\n".join([json.dumps(data_object) for data_object in sample_manifest_lines])

# send manifest to s3
manifest_name = "input.manifest"
s3.Object(input_bucket_name, manifest_name).put(Body=sample_manifest)
input_manifest_uri = f"s3://{input_bucket_name}/{manifest_name}"

# launch our labeling job
time_seconds = int(time.time())
labeling_job_name = f"squad-{time_seconds}"
labeling_job_request = {
    "LabelingJobName": labeling_job_name,
    "HumanTaskConfig": {
        "AnnotationConsolidationConfig": {
            "AnnotationConsolidationLambdaArn": post_annot_lambda_arn,
        },
        "MaxConcurrentTaskCount": 1000,
        "NumberOfHumanWorkersPerDataObject": 1,
        "PreHumanTaskLambdaArn": pre_annot_lambda_arn,
        "TaskAvailabilityLifetimeInSeconds": 864000,
        "TaskDescription": labeling_job_name,
        "TaskTimeLimitInSeconds": 28800,
        "TaskTitle": labeling_job_name,
        "UiConfig": {
            "UiTemplateS3Uri": annotation_ui_s3_uri,
        },
        "WorkteamArn": WORKTEAM_ARN,
    },
    "InputConfig": {
        "DataAttributes": {
            "ContentClassifiers": [
                "FreeOfPersonallyIdentifiableInformation",
                "FreeOfAdultContent",
            ]
        },
        "DataSource": {"S3DataSource": {"ManifestS3Uri": input_manifest_uri}},
    },
    "LabelAttributeName": labeling_job_name,
    # "LabelCategoryConfigS3Uri": label_category_config_uri,
    "OutputConfig": {
        "S3OutputPath": f"s3://{input_bucket_name}/smgt-output",
    },
    "StoppingConditions": {"MaxPercentageOfInputDatasetLabeled": 100},
    "RoleArn": labeler_execution_role_arn,
}
response = sagemaker_cl.create_labeling_job(
    **labeling_job_request
)

response

## Access labeling portal

To access the labeling portal run the following cell and click on the link:

In [ ]:
domain = sagemaker_cl.list_workteams()['Workteams'][0]['SubDomain']
md(f"[labeling worker portal](https://{domain})")

## Check our labeling job status

Once our job has been completed, it may take a few minutes to process and for Ground Truth to put the output in S3. This block will check for the job completion status.

In [ ]:
# labeling_job_name = "squad-1623952684"
labeling_job_description = sagemaker_cl.describe_labeling_job(LabelingJobName=labeling_job_name)
status = labeling_job_description["LabelingJobStatus"]
print(f"Job Status: {status}")
assert status == "Completed"
# labeling_job_description

## Load our labeled data

Once our job is completed that means that our output data is now in S3, we can download it and load into our notebook.

In [25]:
output_manifest = labeling_job_description["LabelingJobOutput"]["OutputDatasetS3Uri"]
output_manifest_lines = s3.Object(*output_manifest[len("s3://"):].split("/", 1)).get()["Body"].read().decode("utf-8").splitlines()
output_manifest_lines = [json.loads(line) for line in output_manifest_lines]

custom_squad_labels_s3_uri = output_manifest_lines[0][labeling_job_name]["s3Uri"]
sample_squad_labels = json.loads(sagemaker.s3.S3Downloader.read_file(custom_squad_labels_s3_uri))

In [ ]:
sample_squad_labels['data'][0]

## Load SQuAD train set 

Let's load SQuAD and add our own labeled examples. SQuAD formats the data as a list of topics. Each topic has it's own set of context statements, each of which have a variety of question and answer pairs. Our sample labels are formatted as their own topic, so we can simply append them to the list of topics already in SQuAD.

In [ ]:
with open('data/v2.0/train-v2.0.json', 'r') as f:
    actual_squad = json.load(f)
    
print('Original length', len(actual_squad['data']))

# add our dataset to squad
actual_squad['data'].extend(sample_squad_labels['data'])
actual_squad['data'][-1]
print('New length', len(actual_squad['data']))

Now we can save our augmented version of SQuAD

In [29]:
with open('data/augmented_squad.json', 'w') as f:
    json.dump(actual_squad, f)

## Create a HF dataset object 

Now that we've combined our data, we can transform it into a Huggingface dataset object. There are several ways to do this. We can use the [load_dataset](https://huggingface.co/docs/datasets/loading_datasets.html) option, in this case we can supply a CSV, JSON, or text file that will then be loaded as a dataset object. You can supply load_dataset a processing script to convert your file into the desired format. In this case for demonstration purposes we are instead going to use the Dataset.from_dict() method which allows us to supply an in-memory dictionary to create a dataset object. 

Run the following block to create our dataset dictionaries.

In [233]:
def create_squad_dict(actual_squad):
    titles = []
    contexts = []
    ids = []
    questions = []
    answers = []
    for example in tqdm(actual_squad["data"]):
        title = example.get("title", "").strip()
        for paragraph in example["paragraphs"]:
            context = paragraph["context"].strip()
            for qa in paragraph["qas"]:
                question = qa["question"].strip()
                id_ = qa["id"]

                answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                answer_list = [answer["text"].strip() for answer in qa["answers"]]
                titles.append(title)
                contexts.append(context)
                questions.append(question)
                ids.append(id_)
                answers.append({
                        "answer_start": answer_starts,
                        "text": answer_list,
                    })

    dataset_dict = {
        "answers":answers,
        "context":contexts,
        "id":ids,
        "question":questions,
        "title":titles,
    }
    return dataset_dict

dataset_dict = create_squad_dict(actual_squad)
test_dataset_dict = create_squad_dict(squad_dev)

100%|██████████| 35/35 [00:00<00:00, 1205.23it/s]


We also need to define our dataset features. We also will define our dataset features. In our case our features are:

* ID - the ID of the text
* Title - the associated title for the topic
* context - the context statement the model must search to find an answer
* question - the question the model is being asked
* answer - the accepted answer text and location in the context statement

HF datasets allow us to define this schema using the Features argument.

In [234]:
squad_dataset = Dataset.from_dict(dataset_dict,
                                 features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "title": datasets.Value("string"),
                    "context": datasets.Value("string"),
                    "question": datasets.Value("string"),
                    "answers": datasets.features.Sequence(
                        {
                            "text": datasets.Value("string"),
                            "answer_start": datasets.Value("int32"),
                        }
                    ),
                    # These are the features of your dataset like images, labels ...
                }
            ))
squad_test = Dataset.from_dict(test_dataset_dict,
                                 features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "title": datasets.Value("string"),
                    "context": datasets.Value("string"),
                    "question": datasets.Value("string"),
                    "answers": datasets.features.Sequence(
                        {
                            "text": datasets.Value("string"),
                            "answer_start": datasets.Value("int32"),
                        }
                    ),
                }
            ))

Once we have our Dataset object created, we then have to tokenize the text. Since models can’t accept raw text as an input, we need to convert our text into a numeric input that it can understand.

In [235]:
def prepare_train_features(examples, tokenizer, max_length, doc_stride):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    pad_on_right = tokenizer.padding_side == "right"

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


## Local training

Let’s start by understanding how training a model in Huggingface works locally, then go over the adjustments we make to run it in SageMaker. 
Huggingface makes training easy through the use of their trainer class. The trainer class allows us to pass in our model, our train and validation datasets, our hyperparameters, and even our tokenizer. Since we already have our model as well as our training and validation sets, we only need to define our hyperparameters. This can be done through the TrainingArguments class. This allows us to specify things like the learning rate, batch size, number of epochs, and more in depth parameters like weight decay or a learning rate scheduling strategy. Once we’ve defined our TrainingArguments, we can pass in our model, our training set, validation set and arguments to instantiate our trainer class. Once this is all ready we can simply call trainer.train() to start training our model. 

To run training locally, you can uncomment out the below block, but this will take a very long time if you are not running on a GPU instance.

In [ ]:
# for running local training, this will likely take a very long time if not running on a GPU instance.

doc_stride=128
max_length=512
tokenized_train = squad_dataset.map(prepare_train_features, batched=True, remove_columns=squad_dataset.column_names, fn_kwargs = {'tokenizer':tokenizer, 'max_length':max_length, 'doc_stride':doc_stride})
tokenized_test = squad_test.map(prepare_train_features, batched=True, remove_columns=squad_test.column_names, fn_kwargs = {'tokenizer':tokenizer, 'max_length':max_length, 'doc_stride':doc_stride})

hf_args = TrainingArguments(
'test_local',
evaluation_strategy = "epoch",
learning_rate=5e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=1,
weight_decay=0.0001,
)

trainer = Trainer(
    model,
    hf_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)

trainer.train()

## Send data to s3

To run the same training job in SageMaker is straightforward. The first step is putting our data in S3 so that our model can access it. SageMaker training allows you to specify a data source, you can use sources like S3, EFS, or FSx for Lustre for high performance data ingestion. In our case, our augmented SQuAD dataset isn’t particularly large, so S3 is a good choice. We upload our training data to a folder in S3 and when SageMaker spins up our training instance, it will download the data from our specified S3 location. 

In [ ]:
!aws s3 cp --recursive data s3://{bucket}/{prefix}

In [166]:
s3train = f's3://{bucket}/{prefix}/'

# create a pointer to our data in S3
train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type=None, s3_data_type='S3Prefix')

data_channels = {'train': train}

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Instantiate the model

Now we are going to instantiate our model, here we are going to specify our hyperparameters for training as well as the number of GPUs we are going to use. For a longer running training job, ml.p4d.24xlarge instances contain 8 A100 GPUs, making them ideal for heavy duty deep learning training. We don't need quite as much horsepower since we are only training our model for a couple epochs, so we can instead use a smaller GPU instance. For our specific training job we are going to use a p3.8xlarge instance consisting of 4 V100 GPUs. 

Once we have set our hyperparameters, we will instantiate a Sagemaker Estimator that we will use to run our training job. We specify the Docker image we just pushed to ECR as well as an entrypoint giving instructions for what operations our container should perform when it starts up. Our Docker container has two commands, train and serve. When we instantiate a training job, behind the scenes Sagemaker is running our Docker container and telling it to run the train command.

In [240]:
account=!aws sts get-caller-identity --query Account --output text

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=!aws configure get region

# metric definition to extract the results
metric_definitions=[
     {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
     {'Name': 'train_samples_per_second', 'Regex': "train_samples_per_second.*=\D*(.*?)$"},
     {'Name': 'epoch', 'Regex': "epoch.*=\D*(.*?)$"},
     {'Name': 'f1', 'Regex': "f1.*=\D*(.*?)$"},
     {'Name': 'exact_match', 'Regex': "exact_match.*=\D*(.*?)$"}]

In [241]:
# hyperparameters, which are passed into the training job
hyperparameters={
    'model_name': model_name,
    'dataset_name':'squad',
    'do_train': True,
    'do_eval': True,
    'fp16': True,
    'train_batch_size': 32, 
    'eval_batch_size': 32,
    'weight_decay':0.01,
    'warmup_steps':500,
    'learning_rate':5e-5,
    'epochs': 2,
    'max_length': 384,
    'max_steps': 100,
    'pad_to_max_length': True,
    'doc_stride': 128,
    'output_dir': '/opt/ml/model'
}

# estimator
huggingface_estimator = HuggingFace(entry_point='run_qa.py',
                                    source_dir='container_training',
                                    metric_definitions=metric_definitions,
                                    instance_type='ml.p3.8xlarge',
                                    instance_count=1,
                                    volume_size=100,
                                    role=role,
                                    transformers_version='4.6.1',
                                    pytorch_version='1.7.1',
                                    py_version='py36',
                                    hyperparameters = hyperparameters)


## Fine-tune the model

If you use an instance with 4 GPUs and a batch size of 32 this process will take ~30 minutes to complete for this particular finetuning task with 2 epochs. Each additional epoch will add another 8 or so minutes. It's recommended to at minimum use a training instance with 4 GPUs, although you will likely get better performance with one of the ml.p3.16xlarge, ml.p3dn.24xlarge, or ml.p4d.24xlarge instances.

We'll be running this training in a SageMaker Training job, which is triggered by the below cell.

In [242]:
huggingface_estimator.fit(data_channels, wait=False, job_name=f'hf-distilbert-squad-{int(time.time())}')

## Download model locally

Once our training job is complete, we can download our trained model. SageMaker stores our model output in S3 as a tarball, so we will need to retrieve it from S3 and then extract the contents locally. 

In [ ]:
# Once training has completed, copy the model weights to our instance.
! aws s3 cp {huggingface_estimator.output_path}{huggingface_estimator.jobs[0].job_name}/output/model.tar.gz model/model.tar.gz

import tarfile

with tarfile.open('model/model.tar.gz', 'r:gz') as f:
    f.extractall('model')

## Load model with trained weights

Now that we've extracted the weights, we can reinitialize our model with them by pointing to the folder containing our weights.

In [237]:
model = AutoModelForQuestionAnswering.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model') # model_name
evl = model.eval()

loading configuration file model/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.2",
  "vocab_size": 30522
}

loading weights file model/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForQuestionAnswering for predictions without

Time to test out our model! Let's try a few different questions from our dev set and see how our model performs:

In [239]:
i = np.random.randint(0,25)
QA_input = {
    'question': sq['paragraphs'][i]['qas'][0]['question'],
    'context': sq['paragraphs'][i]['context']
}
inputs = tokenizer(QA_input['question'], QA_input['context'], return_tensors='pt')
input_ids = inputs["input_ids"].tolist()[0]
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)

answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits
answer_start = torch.argmax(
    answer_start_scores
)  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Context: {QA_input['context']} \n")
print(f"Question: {QA_input['question']} \n")
print(f"Answer: {answer}")

Context: Gamma delta T cells (γδ T cells) possess an alternative T cell receptor (TCR) as opposed to CD4+ and CD8+ (αβ) T cells and share the characteristics of helper T cells, cytotoxic T cells and NK cells. The conditions that produce responses from γδ T cells are not fully understood. Like other 'unconventional' T cell subsets bearing invariant TCRs, such as CD1d-restricted Natural Killer T cells, γδ T cells straddle the border between innate and adaptive immunity. On one hand, γδ T cells are a component of adaptive immunity as they rearrange TCR genes to produce receptor diversity and can also develop a memory phenotype. On the other hand, the various subsets are also part of the innate immune system, as restricted TCR or NK receptors may be used as pattern recognition receptors. For example, large numbers of human Vγ9/Vδ2 T cells respond within hours to common molecules produced by microbes, and highly restricted Vδ1+ T cells in epithelia respond to stressed epithelial cells. 

Qu

## Deploy our trained model

Now that we've finetuned our model, what now? Let's deploy our trained model to an endpoint and ask it some questions!

In [1]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import RealTimePredictor

# this class defines the content type used for our endpoint, in this case plain text
class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')
           

In [ ]:
endpoint_name = 'hf-distilbert-QA-string-endpoint4-185'

# if deploying from a model you trained in the same session 
#     bert_end = torch_model.deploy(instance_type='ml.g4dn.4xlarge', initial_instance_count=1, 
#                           endpoint_name=endpoint_name)

model_data = f"{huggingface_estimator.output_path}{huggingface_estimator.jobs[0].job_name}/output/model.tar.gz"

# We are going to use a SageMaker serving container
torch_model = PyTorchModel(model_data=model_data,
                           source_dir = 'container_serving',
                           role=role,
                          entry_point='transform_script.py',
                          framework_version='1.8.1',
                          py_version='py3',
                          predictor_cls = StringPredictor)
bert_end = torch_model.deploy(instance_type='ml.m5.2xlarge', initial_instance_count=1, #'ml.g4dn.xlarge'
                          endpoint_name=endpoint_name)

Here are a few questions we can ask our model about SageMaker:

In [243]:
amzn_context = """
Amazon SageMaker is a fully managed service that provides every developer and data scientist with the ability to build, train, and deploy machine learning (ML) models quickly. SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high quality models.
Traditional ML development is a complex, expensive, iterative process made even harder because there are no integrated 
tools for the entire machine learning workflow. You need to stitch together tools and workflows, which is time-consuming
and error-prone. SageMaker solves this challenge by providing all of the components used for machine learning in a single 
toolset so models get to production faster with much less effort and at lower cost.
"""

amzn_questions = [
    "How does SageMaker solve traditional ML Development?",
    "What is Traditional ML development?",
]

We can create a predictor object by using the RealTimePredictor class and specifying our endpoint name

In [244]:
# test hf classifier
predictor = RealTimePredictor(endpoint_name=endpoint_name)

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Plain text inference results


In [245]:
%%time

print('Context:',amzn_context)
ind = 0

predictions = []
for amzn_question in amzn_questions:
    print('-------------------------------------------------------------------------------------')
    print('Question:',amzn_question)
    test_text = "|".join((amzn_question, amzn_context))
    pred = predictor.predict(test_text).decode().strip('"')
    print('-------------------------------------------------------------------------------------')
    print('Prediction:',pred)
    print('-------------------------------------------------------------------------------------')
    predictions.append(pred)

Context: 
Amazon SageMaker is a fully managed service that provides every developer and data scientist with the ability to build, train, and deploy machine learning (ML) models quickly. SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high quality models.
Traditional ML development is a complex, expensive, iterative process made even harder because there are no integrated 
tools for the entire machine learning workflow. You need to stitch together tools and workflows, which is time-consuming
and error-prone. SageMaker solves this challenge by providing all of the components used for machine learning in a single 
toolset so models get to production faster with much less effort and at lower cost.

-------------------------------------------------------------------------------------
Question: How does SageMaker solve traditional ML Development?
-----------------------------------------------------------------------------------

## Visualize model results

Now that we've deployed our model and run some inference through it, we can actually visualize the results for multiple questions at once
in the original annotation UI by manually creating some SQuAD format and performing the liquid injection ourselves.

In [246]:
# This is the SquAD input format required by the annotation UI.
squad_txt = json.dumps(
    {
        "version": "v2.0",
        "data": [
            {
                "title": "Ground Truth Marketing",
                "paragraphs": [
                    {
                        "context": amzn_context.replace("\n", ""),
                        "qas": [
                            {
                                "question": question,
                                "id": i,
                                "answers": [
                                    {
                                        "answer_id": i,
                                        "text": prediction,
                                        "answer_start": amzn_context.replace("\n", "").find(prediction),
                                    }
                                ],
                            }
                            for i, (question, prediction) in enumerate(
                                zip(amzn_questions, predictions)
                            )
                        ],
                    },
                ],
            }
        ],
    }
)

with open("qa_annotator.liquid.html") as f:
    # Manually inject our generated squad into the liquid tag.
    tmp_annotator_txt = f.read().replace("{{ task.input.source }}", squad_txt)
    # We'll remove crowd html since we aren't loading within a Ground Truth context.
    tmp_annotator_txt = tmp_annotator_txt.replace('<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>', "")

tmp_annotator_path = "tmp_annotator.html"
with open(tmp_annotator_path, "w") as f:
    f.write(tmp_annotator_txt)

from IPython.display import IFrame
IFrame(tmp_annotator_path, 1280, 800)

## Conclusion

In this notebook, you learned how to create your own question answering dataset using SageMaker Ground Truth and combine it with SQuAD to train your own question answering model using SageMaker training.

Try augmenting SQuAD or even creating an entire dataset with your own questions using SageMaker Ground Truth (https://aws.amazon.com/sagemaker/groundtruth/). Also  try out finetuning different BERT variants using Huggingface (https://huggingface.co/) for question answering. Happy building!


## Cleanup

In [ ]:
!rm bert_base.pt
!rm s3_bucket.txt
bert_end.delete_endpoint()